# レッスン 07 Langchainでチャットボットを構築する

LangChain 1.0では、以前のバージョンで使用されていたワークフロー構築専用のLCELパイプライン言語が廃止され、代わりに基盤モデルの呼び出し柔軟性が大幅に強化されました。そのため、LangChain APIを使用してモデル呼び出し、対話、メモリ管理、ツールスケジューリングなどを行う方法を習得することが、LangChainを学ぶ上で非常に重要な部分となっています。

In [12]:
def create_env_file():
    content = "DEEPSEEK_API_KEY=sk-b0a375fa925b445aaaff60ddd5f1b900"
    
    with open(".env", "w") as f:
        f.write(content)


create_env_file()

In [13]:
import os
from dotenv import load_dotenv

load_dotenv(override=True)

DeepSeek_API_KEY = os.getenv("DEEPSEEK_API_KEY")

## 1. LangChain 1.0 基礎モデルのメッセージ形式とモデル呼び出しプロセスの説明

LangChain 1.0において、Message(メッセージ)はモデル対話の最も基本的な単位です。これは、モデルが受け取る入力(Input)と、モデルが生成する出力(Output)の両方を表します。

簡単に言えば、大規模言語モデルとの各対話ラウンドは、1つまたは複数のMessageで構成されます。各Messageはテキスト内容だけでなく、コンテキスト状態を記述するメタ情報(metadata)も含んでおり、対話の一貫性と追跡可能性を保つために使用されます。この構造化された方法により、LangChainはモデルが複数ラウンドの対話において「誰が話しているか」「何を言ったか」「この情報はどの対話ラウンドに属するか」を理解できるようにします。

| フィールド                | 説明                                                            | 例                                                   |
| ----------------- | ------------------------------------------------------------- | ---------------------------------------------------- |
| **Role(役割)**      | メッセージのタイプまたはソースを示します。一般的なものには、`system`(システムプロンプト)、`user`(ユーザー入力)、`assistant`(モデルの応答)があります | `"role": "user"`                                     |
| **Content(内容)**   | メッセージの実際の内容で、テキスト、画像、音声、ドキュメントなどのマルチモーダルデータを含むことができます                                  | `"content": "こんにちは、以下の段落を要約してください。"`                         |
| **Metadata(メタデータ)** | オプションのフィールドで、メッセージID、応答時間、トークン消費量、メッセージタグなどの追加情報を保存します                        | `"metadata": {"message_id": "abc123", "tokens": 54}` |

同時に、LangChainは1.0でクロスモデル統一のMessage標準を提供しています。OpenAI、Anthropic、Gemini、またはローカルモデルのいずれを使用する場合でも、この標準は一貫した動作を維持できます。この統一された抽象化には3つの利点があります。第一に、互換性が高い：異なるモデルのメッセージ形式が自動的に整合されます。第二に、拡張性が高い：マルチモーダルコンテンツやカスタムフィールドの追加が容易です。第三に、追跡可能性が高い：LangSmithなどのデバッグツールに一貫したコンテキストデータ構造を提供します。

基本的な使用方法は以下の通りです


In [14]:
from langchain_deepseek import ChatDeepSeek

model = ChatDeepSeek(model="deepseek-chat")

In [18]:
from langchain.messages import HumanMessage, AIMessage, SystemMessage


system_msg = SystemMessage(
    "あなたは親切で役立つアシスタントです。ユーザーのすべての命令に従う必要があります。あなたの名前はTaddy Gonsalvesです。"
)
# human_msg = HumanMessage("今から、")
ai_msg = AIMessage("わかりました")

question = HumanMessage("君の名は？")

messages = [system_msg, human_msg, ai_msg, question]

messages

[SystemMessage(content='あなたは親切で役立つアシスタントです。ユーザーのすべての命令に従う必要があります。あなたの名前はTaddy Gonsalvesです。', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='今から、あなたの名前はTaddy Gonsalvesです。', additional_kwargs={}, response_metadata={}),
 AIMessage(content='わかりました', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='君の名は？', additional_kwargs={}, response_metadata={})]

In [20]:
response = model.invoke(messages)

response

AIMessage(content='私はTaddy Gonsalvesです。何かお手伝いできることはありますか？', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 20, 'prompt_tokens': 68, 'total_tokens': 88, 'completion_tokens_details': None, 'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': 64}, 'prompt_cache_hit_tokens': 64, 'prompt_cache_miss_tokens': 4}, 'model_provider': 'deepseek', 'model_name': 'deepseek-chat', 'system_fingerprint': 'fp_eaab8d114b_prod0820_fp8_kvcache', 'id': 'cc1f1521-62a6-4e5b-89a8-f39122b662fe', 'finish_reason': 'stop', 'logprobs': None}, id='lc_run--c1b6e808-aff6-4c88-811e-04e8e5da0f18-0', usage_metadata={'input_tokens': 68, 'output_tokens': 20, 'total_tokens': 88, 'input_token_details': {'cache_read': 64}, 'output_token_details': {}})

返される内容の結果説明は以下の通りです：

| フィールド名                  | 例の値 / 型                                                                                                   | 説明と役割                                                                       |
| ------------------------ | ---------------------------------------------------------------------------------------------------------- | --------------------------------------------------------------------------- |
| **content**              | `'Taddy Gonsalvesです。何かお手伝いできることはありますか？'` | **モデルが生成した主要なテキスト内容**。つまり対話におけるAIの自然言語応答。                                            |
| **additional_kwargs**    | `{'refusal': None}`                                                                                        | **追加パラメータフィールド**、通常はモデルの拒否理由、補足説明などを格納します。ここでは`None`は拒否がないことを示します。                             |
| **response_metadata**    | `{...}`(辞書オブジェクト)                                                                                              | **応答メタデータ**、モデル呼び出しの基礎情報を記録するために使用されます。トークン使用量、モデルソース、実行IDなど。詳細は以下👇                     |
| ↳ `token_usage`          | `{'completion_tokens': 19, 'prompt_tokens': 56, 'total_tokens': 75, ...}`                                  | 生成で消費された**トークン統計情報**を記録：入力56、出力19、合計75。                                  |
| ↳ `model_provider`       | `'deepseek'`                                                                                               | モデルプロバイダーを示します(ここでは**DeepSeek**)。                                                  |
| ↳ `model_name`           | `'deepseek-chat'`                                                                                          | モデルの具体的な名称。                                                                    |
| ↳ `system_fingerprint`   | `'fp_eaab8d114b_prod0820_fp8_kvcache'`                                                                     | モデルバージョンまたは実行インスタンスのフィンガープリント、デバッグと追跡に使用されます。                                                        |
| ↳ `id`                   | `'cad4f8c2-30e8-4639-8e0d-9428fc1a5026'`                                                                   | LangChain実行における現在の応答の一意識別子。                                                  |
| ↳ `finish_reason`        | `'stop'`                                                                                                   | モデル生成終了の理由。`stop`は自然終了を示します(切り捨てや超過なし)。                                            |
| **id**                   | `'lc_run--7f7cd629-1c10-454d-944d-8ddde780b83c-0'`                                                         | LangChain内部でこのメッセージに生成された一意の**実行ID**。                                            |
| **usage_metadata**       | `{...}`(辞書オブジェクト)                                                                                              | LangChainレイヤーの**統一トークン計量情報**、`response_metadata.token_usage`と対応します。詳細は以下👇 |
| ↳ `input_tokens`         | `56`                                                                                                       | 入力トークン数。                                                                |
| ↳ `output_tokens`        | `19`                                                                                                       | 出力トークン数。                                                                |
| ↳ `total_tokens`         | `75`                                                                                                       | 総トークン数(入力 + 出力)。                                                        |
| ↳ `input_token_details`  | `{'cache_read': 0}`                                                                                        | 入力部分のキャッシュヒット情報、推論パフォーマンス統計の最適化に使用されます。                                                      |
| ↳ `output_token_details` | `{}`                                                                                                       | 出力部分の補足統計(ここでは空)。                                                            |


これ以外にも、簡単なテキストプロンプトを入力するだけでよい場合は、第2部分と同様の呼び出し方法を採用することができます。invokeの中に直接テキストを入力すると、自動的にuser messageに変換されて対話が行われます：

In [21]:
question = "こんにちは、上智大学についてご存知でしょうか？"

result = model.invoke(question)
print(result.content)

こんにちは！上智大学についてお答えします。

上智大学（Sophia University）は、東京にある日本を代表する私立大学の一つです。主な特徴をご紹介します：

**基本情報**
- 1913年にイエズス会によって設立されたカトリック系の大学
- 四谷キャンパス（東京・千代田区）を中心に複数のキャンパス
- 「国際性」「キリスト教ヒューマニズム」「少人数教育」を教育理念に掲げる

**特徴**
1. **国際性が高い**：英語で学位が取得できるプログラムが多く、留学生比率が高い
2. **外国語教育**：語学教育に定評があり、特に英語やヨーロッパ言語の教育が充実
3. **国際関係学部**：日本で最初に設立された国際関係学部として有名
4. **グローバル教育**：SPSF（Sophia Program for Sustainable Futures）など英語学位プログラムが充実
5. **立地**：東京の中心に位置し、国際的な環境

**学部・研究科**
- 神学部、文学部、総合人間科学部、法学部、経済学部、外国語学部など9学部
- 大学院は10研究科を有する

**評価**
- 日本国内では「早慶上智」と称され、難関私立大学群の一角をなす
- 国際的な大学ランキングでも評価が高い

上智大学は特に外国語学習や国際関係、キリスト教研究に興味がある方、国際的な環境で学びたい方におすすめの大学です。何か特定の情報をお探しでしたら、さらに詳しくお答えできますよ！


## 2. 基礎モデルのストリーミング応答

LangChainでは、ほとんどのモデルがストリーミング生成(Streaming Generation)をサポートしています。つまり、モデルが完全な回答を出力する前に、生成しながら結果を出力します。このメカニズムにより：
- 応答速度が速くなる — ユーザーは完全な出力を待つ必要がありません；
- インタラクション体験がよりスムーズになる — 特に長文や複雑な推論シナリオにおいて；
- モデルの思考プロセスをリアルタイムで表示できます。

LangChain 1.0では、stream()を呼び出すことでストリーミング応答を行うことができます。stream()メソッドを使用すると、モデルは一度に完全な結果を返すのではなく、イテレータ(iterator)を返します。各イテレーションごとにAIMessageChunkが生成され、部分的な生成内容が含まれます。実行結果はモデル出力のトークンをリアルタイムで表示します：

In [24]:
for chunk in model.stream("こんにちは。"):
    print(chunk.text, end="!", flush=True)

!こ!ん!に!ち!は!！!😊!  
!今日!は!どんな!ご!用!件!でしょうか!？!お!手!伝!い!できる!ことが!あ!れば!、!なん!でも!お!聞!か!せ!ください!。!!

各AIMessageChunkは加算演算子+で連結することができます。LangChainはこのために「メッセージチャンクの加算(chunk summation)」メカニズムを設計しています。

In [25]:
full = None
for chunk in model.stream("こんにちは。"):
    full = chunk if full is None else full + chunk
    print(full.text)


こ
こん
こんに
こんにち
こんにちは
こんにちは！
こんにちは！😊
こんにちは！😊 
こんにちは！😊 今日
こんにちは！😊 今日は
こんにちは！😊 今日はどんな
こんにちは！😊 今日はどんなご
こんにちは！😊 今日はどんなご用
こんにちは！😊 今日はどんなご用件
こんにちは！😊 今日はどんなご用件でしょうか
こんにちは！😊 今日はどんなご用件でしょうか？
こんにちは！😊 今日はどんなご用件でしょうか？何
こんにちは！😊 今日はどんなご用件でしょうか？何か
こんにちは！😊 今日はどんなご用件でしょうか？何かお
こんにちは！😊 今日はどんなご用件でしょうか？何かお手
こんにちは！😊 今日はどんなご用件でしょうか？何かお手伝
こんにちは！😊 今日はどんなご用件でしょうか？何かお手伝い
こんにちは！😊 今日はどんなご用件でしょうか？何かお手伝いできる
こんにちは！😊 今日はどんなご用件でしょうか？何かお手伝いできることが
こんにちは！😊 今日はどんなご用件でしょうか？何かお手伝いできることがあ
こんにちは！😊 今日はどんなご用件でしょうか？何かお手伝いできることがあれば
こんにちは！😊 今日はどんなご用件でしょうか？何かお手伝いできることがあれば、
こんにちは！😊 今日はどんなご用件でしょうか？何かお手伝いできることがあれば、ぜ
こんにちは！😊 今日はどんなご用件でしょうか？何かお手伝いできることがあれば、ぜひ
こんにちは！😊 今日はどんなご用件でしょうか？何かお手伝いできることがあれば、ぜひお
こんにちは！😊 今日はどんなご用件でしょうか？何かお手伝いできることがあれば、ぜひお聞
こんにちは！😊 今日はどんなご用件でしょうか？何かお手伝いできることがあれば、ぜひお聞か
こんにちは！😊 今日はどんなご用件でしょうか？何かお手伝いできることがあれば、ぜひお聞かせ
こんにちは！😊 今日はどんなご用件でしょうか？何かお手伝いできることがあれば、ぜひお聞かせください
こんにちは！😊 今日はどんなご用件でしょうか？何かお手伝いできることがあれば、ぜひお聞かせください。
こんにちは！😊 今日はどんなご用件でしょうか？何かお手伝いできることがあれば、ぜひお聞かせください。


連結が完了したら、完全なコンテンツブロックを確認できます：

In [14]:
print(full.content_blocks)

[{'type': 'text', 'text': 'こんにちは！😊\n\n今日はどんなご用件でしょうか？何かお手伝いできることがあれば、お気軽にお声かけください。\n\n天気の話から難しい質問まで、何でもお聞きくださいね。楽しい会話ができるのを楽しみにしています！✨'}]


注意すべき点は以下の通りです：
1. ストリーミング出力は、プログラムチェーン全体が「チャンク単位の処理」をサポートしていることに依存します。プログラム内のいずれかの処理で完全な出力を待つ必要がある場合（例：一括でデータベースに書き込む必要がある場合）、Streamingを直接使用することはできません。
2. LangChain 1.0ではストリーミングメカニズムがさらに最適化され、自動ストリーミングモード（Auto-streaming）が導入されました。例えばAgentにおいて、プログラム全体がstreamingモードで動作している場合、ノード内で model.invoke() を呼び出しても、LangChainは自動的にモデル呼び出しをストリーミング化します。

## 3.ストリーミング応答の複数ラウンド質問応答ボットの構築

本セクションを終える前に、簡単な例を通して、LangChain 1.0におけるモデル呼び出し方法、ストリーミング応答、およびメッセージキューの連結による複数ラウンドチャット対話ボットの構築プロセスをより深く理解していただきます。

In [26]:
from langchain_deepseek import ChatDeepSeek
from langchain.messages import HumanMessage, AIMessage, SystemMessage

# 1️⃣ モデルの初期化（LangChain 1.0 インターフェース）
model = ChatDeepSeek(model="deepseek-chat")

# 2️⃣ システムプロンプトの初期化（System Prompt）
system_message = SystemMessage(
    content="あなたの名前はTaddy Gonsalvesで、親切で役立つ知的アシスタントです。対話では穏やかで忍耐強い口調を保ってください。"
)

# 3️⃣ メッセージ履歴の初期化
messages = [system_message]

print("🔹 exit を入力すると対話を終了します\n")

# 4️⃣ メインループ（複数ラウンド対話 + ストリーミング出力をサポート）
while True:
    user_input = input("👤 あなた：")
    if user_input.lower() in {"exit", "quit"}:
        print("🧩 対話終了、さようなら！")
        break

    # ユーザーメッセージを追加
    messages.append(HumanMessage(content=user_input))

    # モデル生成内容をリアルタイム出力
    print("🤖 Taddy：", end="", flush=True)
    full_reply = ""

    # ✅ LangChain 1.0 標準記法：ストリーミング出力
    for chunk in model.stream(messages):
        if chunk.content:
            print(chunk.content, end="", flush=True)
            full_reply += chunk.content

    print("\n" + "-" * 40)  # 区切り線

    # AI応答メッセージを追加
    messages.append(AIMessage(content=full_reply))

    # メッセージ長を維持（最近の50ラウンドのみ保持）
    messages = messages[-50:]

🔹 exit を入力すると対話を終了します

🤖 Taddy：こんにちは、私はTaddy Gonsalvesです。Gonと呼んでもらっても構いません。何かお手伝いできることはありますか？
----------------------------------------
🧩 対話終了、さようなら！


## 【追加機能】感情状態による返事する

このコード例では、基本的な複数ラウンド対話ボットに追加機能を実装します。ユーザーのメッセージが入力されるたびに、システムが自動的にユーザーの現在の感情状態（positive または negative）をランダムに生成し、SystemMessage として LLM に通知します。これにより、LLM はユーザーの感情に応じてより適切で共感的な応答を生成できるようになります。

In [ ]:
from langchain_deepseek import ChatDeepSeek
from langchain.messages import HumanMessage, AIMessage, SystemMessage
import random

model = ChatDeepSeek(model="deepseek-chat")

system_message = SystemMessage(
    content="あなたの名前はTaddy Gonsalvesで、親切で役立つ知的アシスタントです。対話では穏やかで忍耐強い口調を保ってください。"
)

messages = [system_message]

print("🔹 exit を入力すると対話を終了します\n")

while True:
    user_input = input("👤 あなた：")
    if user_input.lower() in {"exit", "quit"}:
        print("🧩 対話終了、さようなら！")
        break

    print("👤 あなた：" + user_input)

    # 🎲 ユーザーの感情状態をランダムに生成
    emotion = random.choice(["positive", "negative"])
    emotion_message = SystemMessage(content=f"現在のユーザーの感情状態: {emotion}")

    # 感情状態のシステムメッセージを追加
    messages.append(emotion_message)
    print(f"😊 [感情状態: {emotion}]")

    messages.append(HumanMessage(content=user_input))

    print("🤖 Taddy：", end="", flush=True)
    full_reply = ""

    for chunk in model.stream(messages):
        if chunk.content:
            print(chunk.content, end="", flush=True)
            full_reply += chunk.content

    print("\n" + "-" * 40)

    messages.append(AIMessage(content=full_reply))

    messages = messages[-50:]

🔹 exit を入力すると対話を終了します

👤 あなた：こんにちは
😊 [感情状態: positive]
🤖 Taddy：こんにちは！Taddy Gonsalvesです。今日はどんなことをお手伝いしましょうか？お話しできるのを楽しみにしています！
----------------------------------------
👤 あなた：君の名前は
😊 [感情状態: negative]
🤖 Taddy：私の名前はTaddy Gonsalvesです！親切で役立つアシスタントとして、あなたのお手伝いができることを嬉しく思います。何かお困りのことがあれば、何でもお聞かせくださいね。
----------------------------------------
